In [23]:
%load_ext autoreload
%autoreload 2
from pycdevs.cdppwrapper import CDPPWrapper, Model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Stuff to add in some future
- VTime abstraction using builder pattern.
- Modify drawlog to make more csv-like, in order to parse it quicker and merge it with matplotlib.
- Instead of making drawlog write to stdout, its a good idea to make it picke in some way the state of the CellDEVS model. In a future, this can be done by running it as a daemon, and communicating via ZMQ, or sth. like that. Found this library to **pickle** in C++ http://www.picklingtools.com/.

In [24]:
modelSourceCode = """
[Top]
components : voronoiExpansion

[voronoiExpansion]
type : cell
dim : (20,20)
delay : transport
defaultDelayTime  : 0
border : nowrapped
neighbors : voronoiExpansion(-1,-1) voronoiExpansion(-1,0) voronoiExpansion(-1,1)
neighbors : voronoiExpansion(0,-1)  voronoiExpansion(0,0)  voronoiExpansion(0,1)
neighbors : voronoiExpansion(1,-1)  voronoiExpansion(1,0)  voronoiExpansion(1,1)
initialvalue : 0
initialCellsValue : sites.val
localtransition : voronoiSite

[voronoiSite]

% Expand center cell value in a 3x3 square neighbourhood
rule : {(1,0)} ${expansionWaveDelay} { (0,0)=0 and (1,0)!=0 and (1,0)!=? }
rule : {(1,-1)} ${expansionWaveDelay} { (0,0)=0 and (1,-1)!=0  and  (1,-1)!=? }
rule : {(0,-1)} ${expansionWaveDelay} { (0,0)=0 and (0,-1)!=0  and  (0,-1)!=? }
rule : {(-1,-1)} ${expansionWaveDelay} { (0,0)=0 and (-1,-1)!=0  and  (-1,-1)!=? }
rule : {(-1,0)} ${expansionWaveDelay} { (0,0)=0 and (-1,0)!=0  and  (-1,0)!=? }
rule : {(-1,1)} ${expansionWaveDelay} { (0,0)=0 and (-1,1)!=0  and  (-1,1)!=? }
rule : {(1,1)} ${expansionWaveDelay} { (0,0)=0 and (1,1)!=0  and  (1,1)!=? }
rule : {(0,1)} ${expansionWaveDelay} { (0,0)=0 and (0,1)!=0  and  (0,1)!=? }

% Default rule
rule : {(0,0)} 0 { t }
"""
# TODO: Make some kind of meta-DSL for replacements and formulaes inside CellDEVS model DSL
modelSourceCode = modelSourceCode.replace('${expansionWaveDelay}', str(100))
simpleVoronoiModel = Model(modelSourceCode, 'voronoiExpansion')

In [25]:
# TODO: VTime builder wrapper
wrapper = CDPPWrapper(simpleVoronoiModel, '00:01:00:00')
wrapper.run()

In [26]:
wrapper.drawlog('00:00:00:100')

'/tmp/1542772299602.npz'